In [ ]:
import pandas as pd
import json
import os
import geopandas as gpd
import requests
import time
import googlemaps
import folium
from shapely.geometry import Polygon
import numpy as np
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
from scipy.spatial import distance
from scipy.spatial import cKDTree
import re
from datetime import datetime, timedelta
from statistics import mean
from datetime import datetime

In [ ]:
api_key = os.getenv('GOOGLE_MAPS_API_KEY')

In [ ]:
df = pd.read_csv('data/grid_coordinates.csv')

coordinates = list(map(tuple, df.values))

coordinates

In [ ]:
gmaps = googlemaps.Client(key=api_key)

# Define a function to retrieve all venues for a given location
def get_all_venues(location, radius, venue_type):
    initial_result = gmaps.places_nearby(location=location, radius=radius, type=venue_type)
    all_results = initial_result.get('results', [])

    while 'next_page_token' in initial_result:
        time.sleep(3)  # Necessary sleep before using the next page token
        initial_result = gmaps.places_nearby(page_token=initial_result['next_page_token'])
        all_results.extend(initial_result.get('results', []))

    return all_results

# Initialize an empty list to store all venues
transit_locations = []

transit_types = ['bus_station', 'bus_stop', 'ferry_terminal', 'heliport', 'light_rail_station', 'park_and_ride', 'subway_station', 'train_station', 'transit_depot', 'transit_station']

# Loop through each coordinate
for lat, lng in coordinates:
    location = f'{lat},{lng}'  # Format the location as a string
    venues = get_all_venues(location, radius=762, venue_type=transit_types)
    transit_locations.extend(venues)

    # Respect API rate limits
    time.sleep(3)  # Adjust the delay as per the API's rate limit

In [ ]:
transit_places_result = pd.DataFrame(transit_locations)

transit_places_result.to_csv('data/transit_places_result.csv')

In [ ]:
transit_places_result.head()

In [ ]:
transit_df = transit_places_result[['name', 'place_id', 'geometry', 'types']]

In [ ]:
transit_df['types']

In [ ]:
transit_df['geometry'][0]['location']['lat']

In [ ]:
# Create a map centered around the average coordinates of the venues
average_lat = sum(place['geometry']['location']['lat'] for place in transit_df) / len(transit_df)
average_lng = sum(place['geometry']['location']['lng'] for place in transit_df) / len(transit_df)
m = folium.Map(location=[average_lat, average_lng], zoom_start=13)

# Add markers to the map for each venue
for place in transit_df:
    lat = place['geometry']['location']['lat']
    lng = place['geometry']['location']['lng']
    name = place.get('name', 'No Name')
    folium.Marker([lat, lng], popup=name).add_to(m)

# Display the map
m

In [ ]:
import pandas as pd
import folium

# Assuming transit_df is a pandas DataFrame and the 'geometry' column contains dictionaries

# Extract 'lat' and 'lng' into separate columns in the DataFrame
transit_df['lat'] = transit_df['geometry'].apply(lambda x: x['location']['lat'])
transit_df['lng'] = transit_df['geometry'].apply(lambda x: x['location']['lng'])

# Calculate the average latitude and longitude
average_lat = transit_df['lat'].mean()
average_lng = transit_df['lng'].mean()

# Create a map centered around the average coordinates of the venues
m = folium.Map(location=[average_lat, average_lng], zoom_start=13)

# Add markers to the map for each venue
for _, place in transit_df.iterrows():
    lat = place['lat']
    lng = place['lng']
    name = place.get('name', 'No Name')  # Adjust this line if 'name' is nested or not directly in the DataFrame
    folium.Marker([lat, lng], popup=name).add_to(m)

# Display the map
m

